In [1]:
import time, math, os
from tqdm import tqdm
import gc
import pickle
import random
from datetime import datetime
from operator import itemgetter
import numpy as np
import pandas as pd
import warnings
import collections
from collections import defaultdict
warnings.filterwarnings('ignore')

In [3]:
# 节约内存的一个标配函数
def reduce_mem(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'
          .format(end_mem,100*(start_mem-end_mem)/start_mem,(time.time()-starttime)/60))
    return df

# 新闻推荐

数据来自某新闻APP平台的用户交互数据：
- 包括<mark>30万用户，近300万次点击log
- 共36万多篇不同的新闻文章，同时<mark>每篇新闻文章有对应的embedding向量表示。

为了保证比赛的公平性，将会从中
- 抽取20万用户的点击日志数据作为<mark>训练集，
- 5万用户的点击日志数据作为<mark>测试集A，
- 5万用户的点击日志数据作为<mark>测试集B。

In [2]:
data_path = './data/'
save_path = './results/'

In [4]:
# debug模式：从训练集中划出一部分数据来调试代码
def get_all_click_sample(data_path, sample_nums=10000):
    """
        训练集中采样一部分数据调试
        data_path: 原数据的存储路径
        sample_nums: 采样数目（这里由于机器的内存限制，可以采样用户做）
    """
    all_click = pd.read_csv(data_path + 'train_click_log.csv')
    all_user_ids = all_click.user_id.unique() #train_click_log中所有去重的user_id

    sample_user_ids = np.random.choice(all_user_ids, size=sample_nums, replace=False) #选一部分user_id作为训练集
    all_click = all_click[all_click['user_id'].isin(sample_user_ids)]
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

# 读取点击数据，这里分成线上和线下，如果是为了获取线上提交结果应该讲测试集中的点击数据合并到总的数据中
# 如果是为了线下验证模型的有效性或者特征的有效性，可以只使用训练集
def get_all_click_df(data_path='./data/', offline=True):
    if offline:
        all_click = pd.read_csv(data_path + 'train_click_log.csv')
    else:
        trn_click = pd.read_csv(data_path + 'train_click_log.csv')
        tst_click = pd.read_csv(data_path + 'testA_click_log.csv')
        all_click = trn_click.append(tst_click)
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

In [5]:
# 全量训练集
all_click_df = get_all_click_df(offline=False)

## 赛题介绍

正确了解赛题背后的思想以及赛题业务逻辑的清晰，有利于花费更少时间构建更为有效的特征模型：
- 目的：<mark>根据用户历史浏览点击新闻文章的数据信息，预测用户的最后一次点击的新闻文章<mark>（用户未来的点击行为）

这道赛题的设计初衷是引导大家了解推荐系统中的一些业务背景， 解决实际问题。

## 读取采样或全量数据

#### train_click_log（都是数值）:

| user_id| click_article_id | click_timestamp| click_environment|click_deviceGroup|
| -- | --  | -- | -|--|
| user_id| 点击文章id | 点击时间戳| 点击环境|点击设备组|

|click_os|click_country|click_region|click_referrer_type|
|--|--|--|--|
|点击操作系统|国家|地区|点击来源类型|


#### article
| article_id|category_id|created_at_ts|words_count| emb_1,emb_2,…,emb_249 |
| --|--|--|--| --|
| 文章id|文章类别id|文章创建时间戳|文章字数|文章embedding向量表示|

## 评价方式理解

理解评价方式， 我们需要结合着最后的提交文件来看， 根据sample.submit.csv， 我们最后提交的格式是针对每个用户， 我们都会给出五篇文章的推荐结果，按照点击概率从前往后排序。 

而真实的每个用户最后一次点击的文章只会有一篇的真实答案， 所以我们就看我们推荐的这五篇里面是否有命中真实答案的。

比如对于user1来说， 我们的提交会是：

    user1, article1, article2, article3, article4, article5.

<mark>评价指标的公式如下：
    score(user)=[5∑k=1] s(user,k)/k



- 假如article1就是真实的用户点击文章，也就是article1命中， 则s(user1,1)=1, s(user1,2-4)都是0， 
- 如果article2是用户点击的文章， 则s(user,2)=1/2,s(user,1,3,4,5)都是0。
- 也就是score(user)=命中第几条的倒数。
- 如果都没中， 则score(user1)=0。 

这个是合理的， 因为我们希望的就是命中的结果尽量靠前， 而此时分数正好比较高。

## 赛题理解

与普通数据挖掘的区别：
- 目标上， 要预测最后一次点击的新闻文章，也就是我们给用户推荐的是新闻文章， 并不是普通的回归或分类；
- 数据上， 也不是我们之前遇到的那种特征+标签的数据，而是基于了真实的业务场景， 拿到的用户的点击日志；

那么，如何把该预测问题转成一个监督学习的问题(特征+标签)，然后进行ML，DL等建模预测？
- 如何转成一个监督学习问题呢？ 
- 转成一个什么样的监督学习问题呢？ 
- 我们能利用的特征又有哪些呢？ 
- 又有哪些模型可以尝试呢？ 
- 此次面对数万级别的文章推荐，我们又有哪些策略呢？

=> 从36万篇文章中预测某一篇的话,可能是一个多分类的问题(36万类里面选1)，但太过庞大；

=> <mark>预测出某个用户最后一次对于某一篇文章会进行点击的概率 CTR，概率最大的那篇文章就是用户最后一次可能点击的新闻文章 </mark>; 这样就把原问题变成了一个点击率预测的问题(用户, 文章) --> 点击的概率(软分类),对于模型的选择就基本上有大致方向了，比如最简单的逻辑回归模型。

### 分类问题 CTR预估

大致的解决思路：
- 转成一个<mark>分类问题;
- 分类的<mark>标签 Y 就是用户是否会点击某篇文章//二分类;
- 分类问题的<mark>特征 X 如用户和文章;
- 训练一个<mark>分类模型， 对某用户最后一次点击某篇文章的概率进行预测;

那么又会有几个问题：
- 如何转成监督学习问题？ 
- 训练集和测试集怎么制作？ 
- 我们又能利用哪些特征？ 
- 我们又可以尝试哪些模型？ 
- 面对36万篇文章， 20多万用户的推荐， 我们又有哪些策略来缩减问题的规模？
- 如何进行最后的预测？

### 获取 用户-文章-点击事件 字典

In [8]:
# 根据点击时间获取用户的点击文章序列   {user1: [(item1, time1), (item2, time2)..]...}
def get_user_item_time(click_df):
    
    click_df = click_df.sort_values('click_timestamp')
    
    def make_item_time_pair(df):
        return list(zip(df['click_article_id'], df['click_timestamp']))
    
    user_item_time_df = click_df.groupby('user_id')['click_article_id', 'click_timestamp']\
                                                            .apply(lambda x: make_item_time_pair(x))\
                                                            .reset_index().rename(columns={0: 'item_time_list'})
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    
    return user_item_time_dict

In [25]:
# pandas 例子
tmp_df = pd.DataFrame({"class":[1,2,1,2,1,2],
                       "idx":[1,2,3,4,5,6],
                       "标号":["一","二","三","四","五","六"]
})
dict(zip(tmp_df["idx"],tmp_df["标号"]))

{1: '一', 2: '二', 3: '三', 4: '四', 5: '五', 6: '六'}

### 获取点击最多的TopK个文章

In [24]:
# 获取近期点击最多的文章
def get_item_topk_click(click_df, k):
    topk_click = click_df['click_article_id'].value_counts().index[:k]
    return topk_click

### itemCF的物品相似度计算

In [39]:
tmp_dict = dict()
tmp_dict.setdefault("i", {})
tmp_dict["i"].setdefault("j", 0)
print(tmp_dict," and ", tmp_dict["i"]["j"])

{'i': {'j': 0}}  and  0


In [42]:
def itemcf_sim(df):
    """
        文章与文章之间的相似性矩阵计算
        :param df: 数据表
        :item_created_time_dict:  文章创建时间的字典
        return : 文章与文章的相似性矩阵
        思路: 基于物品的协同过滤， 在多路召回部分会加上关联规则的召回策略
    """
    user_item_time_dict = get_user_item_time(df) #{user1: [(item1, time1), (item2, time2)..]...}
    
    # 计算物品相似度
    i2i_sim = {}
    item_cnt = defaultdict(int)#{item_i:{item_j:Wij}}
    
    for user, item_time_list in tqdm(user_item_time_dict.items()): #tqdm(iterator)在长循环中添加一个进度提示信息
        # 在基于商品的协同过滤优化的时候可以考虑时间因素
        for i, i_click_time in item_time_list:
            item_cnt[i] += 1 #item总共被点击几次
            i2i_sim.setdefault(i, {}) #若不存在，创建一个dict element:{item_i:{}}
            for j, j_click_time in item_time_list:
                if(i == j):
                    continue
                i2i_sim[i].setdefault(j, 0) 
                '''{item_i:{item_j:0}} = item_i和item_j的相似性分子，同时被一个user点击，+1'''
                i2i_sim[i][j] += 1 / math.log(len(item_time_list) + 1)
                
    i2i_sim_ = i2i_sim.copy()
    for i, related_items in i2i_sim.items(): #对每个item_pair除以分母（item_i总点击次数，根号，同item_j,相乘）
        for j, wij in related_items.items():
            i2i_sim_[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])
    
    # 将得到的相似性矩阵保存到本地
    pickle.dump(i2i_sim_, open(save_path + 'itemcf_i2i_sim.pkl', 'wb')) 
    #pickle.dump：将对象obj保存到文件file中去，“write binary”
    
    return i2i_sim_

In [41]:
i2i_sim = itemcf_sim(all_click_df)

100%|████████████████████████████████████████████████████████████████████████| 250000/250000 [00:37<00:00, 6623.98it/s]


## itemCF 的文章推荐

In [43]:
# 基于商品的召回i2i
def item_based_recommend(user_id, user_item_time_dict, i2i_sim, sim_item_topk, recall_item_num, item_topk_click):
    """
        基于文章协同过滤的召回
        :param user_id: 用户id
        :param user_item_time_dict: 字典,{user1: [(item1, time1), (item2, time2)..]...}
        :param i2i_sim: 字典，文章相似性矩阵, {item_i:{item_j:Wij}}
        :param sim_item_topk: 整数， 选择与当前文章最相似的前k篇文章
        :param recall_item_num: 整数， 最后的召回文章数量
        :param item_topk_click: 列表，点击次数最多的文章列表，用户召回补全        
        return: 召回的文章列表 [item1:score1, item2: score2...]
        注意: 基于物品的协同过滤， 在多路召回部分会加上关联规则的召回策略
    """
    
    # 获取用户历史交互的文章
    user_hist_items = user_item_time_dict[user_id] # 注意，此时获取得到的是一个元组列表，需要将里面的user_id提出来
    user_hist_items_ = {item_id for item_id, _ in user_hist_items} #此用户点击过的{item_id, ....}

    item_rank = {}
    for loc, (i, click_time) in enumerate(user_hist_items): #遍历用户点击过的所有item，找每个item的相似文章
        for j, wij in sorted(i2i_sim[i].items(), key=lambda x: x[1], reverse=True)[:sim_item_topk]:
        #对于每一个点击的文章item_i, 按Wij排序，遍历topK 与i最相似的前k篇文章, 加入到推荐列表
            if j  in user_hist_items_:
                continue       
            item_rank.setdefault(j, 0)
            item_rank[j] +=  wij
    
    # 不足10个，用热门商品补全
    if len(item_rank) < recall_item_num:
        for i, item in enumerate(item_topk_click):
            if item in item_rank.items(): # 填充的item应该不在原来的列表中
                continue
            item_rank[item] = - i - 100 # 随便给个负数就行，排在推荐列表的最后面
            if len(item_rank) == recall_item_num:
                break
    
    item_rank = sorted(item_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num] #最后的召回文章
        
    return item_rank

### 给每个用户根据物品的协同过滤推荐文章

In [44]:
# 定义
user_recall_items_dict = collections.defaultdict(dict)

# 获取 用户 - 文章 - 点击时间的字典 => {user1: [(item1, time1), (item2, time2)..]...}
user_item_time_dict = get_user_item_time(all_click_df)

# 去取文章相似度 => {item_i:{item_j:Wij, ...}}
i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim.pkl', 'rb'))

# 相似文章的数量
sim_item_topk = 10

# 召回文章数量
recall_item_num = 10

# 用户热度补全
item_topk_click = get_item_topk_click(all_click_df, k=50)

for user in tqdm(all_click_df['user_id'].unique()): #遍历每一个有过点击行为的user，推荐 n个文章
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, i2i_sim, 
                                                        sim_item_topk, recall_item_num, item_topk_click)
                                   #return item_rank = dict({item_recom: score, ...})

100%|██████████████████████████████████████████████████████████████████████████| 250000/250000 [53:46<00:00, 77.48it/s]


### 召回字典转换成df

In [45]:
# 将字典的形式转换成df
user_item_score_list = []

for user, items in tqdm(user_recall_items_dict.items()): # {user:{item_recom: score, ...},  ...}
    for item, score in items:
        user_item_score_list.append([user, item, score])

recall_df = pd.DataFrame(user_item_score_list, columns=['user_id', 'click_article_id', 'pred_score'])

100%|███████████████████████████████████████████████████████████████████████| 250000/250000 [00:08<00:00, 30787.85it/s]


### 生成提交文件

In [46]:
# 生成提交文件
def submit(recall_df, topk=5, model_name=None):
    recall_df = recall_df.sort_values(by=['user_id', 'pred_score'])
    recall_df['rank'] = recall_df.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    # rank over(partition by 'user_id', order by 'pred_score')
    
    # 判断是不是每个用户都有5篇文章及以上
    tmp = recall_df.groupby('user_id').apply(lambda x: x['rank'].max())
    assert tmp.min() >= topk
    
    #df['user_id', 'click_article_id', 'pred_score','rank']
    del recall_df['pred_score']
    #将 ['user_id', 'rank'] 作为索引 #unstack把 相同的user_id的数据 摞起来：故一行有5个rank顺序的article_id
    submit = recall_df[recall_df['rank'] <= topk].set_index(['user_id', 'rank']).unstack(-1).reset_index()
    
    submit.columns = [int(col) if isinstance(col, int) else col for col in submit.columns.droplevel(0)]
    # 按照提交格式定义列名
    submit = submit.rename(columns={'': 'user_id', 1: 'article_1', 2: 'article_2', 
                                                  3: 'article_3', 4: 'article_4', 5: 'article_5'})
    
    save_name = save_path + model_name + '_' + datetime.today().strftime('%m-%d') + '.csv'
    submit.to_csv(save_name, index=False, header=True)

In [47]:
# 获取测试集
tst_click = pd.read_csv(data_path + 'testA_click_log.csv')
tst_users = tst_click['user_id'].unique()

# 从所有的召回数据中将测试集中的用户选出来
tst_recall = recall_df[recall_df['user_id'].isin(tst_users)]

# 生成提交文件
submit(tst_recall, topk=5, model_name='itemcf_baseline')